# Day 17 - LSTM and GRU

## Overview

In this notebook, we’ll build and compare LSTM and GRU architectures on the IMDB sentiment analysis dataset.
Both are designed to handle long-term dependencies better than simple RNNs.

## Import Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, preprocessing, callbacks #type: ignore
import matplotlib.pyplot as plt

tf.random.set_seed(42)

## Load and Preprocess Data

In [2]:
vocab_size = 10000
max_len = 200

(x_train, y_train), (x_test, y_test) = datasets.imdb.load_data(num_words=vocab_size)
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

print(f"Training samples: {len(x_train)}, Test samples: {len(x_test)}")

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Training samples: 25000, Test samples: 25000


## Build LSTM and GRU Models

In [4]:
def build_lstm():
    model = models.Sequential([
        layers.Embedding(vocab_size, 128, input_length=max_len),
        layers.LSTM(128, return_sequences=False),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_gru():
    model = models.Sequential([
        layers.Embedding(vocab_size, 128, input_length=max_len),
        layers.GRU(128, return_sequences=False),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

lstm_model = build_lstm()
gru_model = build_gru()

lstm_model.summary()
gru_model.summary()

d:\Deep Learning and Neural Networks\.deep_learn\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Compile and Train

In [5]:
def train_model(model, name):
    print(f"\nTraining {name} model...")
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    cb = [callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)]
    history = model.fit(x_train, y_train,
                        validation_data=(x_test, y_test),
                        epochs=10, batch_size=128, callbacks=cb)
    loss, acc = model.evaluate(x_test, y_test)
    print(f"{name} Test Accuracy: {acc*100:.2f}%\n")
    return history

hist_lstm = train_model(lstm_model, 'LSTM')
hist_gru = train_model(gru_model, 'GRU')



Training LSTM model...
Epoch 1/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 124s 615ms/step - accuracy: 0.7824 - loss: 0.4418 - val_accuracy: 0.8396 - val_loss: 0.3635
Epoch 2/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 120s 614ms/step - accuracy: 0.8956 - loss: 0.2615 - val_accuracy: 0.8678 - val_loss: 0.3318
Epoch 3/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 127s 651ms/step - accuracy: 0.9149 - loss: 0.2199 - val_accuracy: 0.8573 - val_loss: 0.4055
Epoch 4/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 121s 617ms/step - accuracy: 0.9319 - loss: 0.1764 - val_accuracy: 0.8629 - val_loss: 0.3645
Epoch 5/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 113s 577ms/step - accuracy: 0.9246 - loss: 0.1994 - val_accuracy: 0.8370 - val_loss: 0.4559
782/782 ━━━━━━━━━━━━━━━━━━━━ 53s 67ms/step - accuracy: 0.8678 - loss: 0.3318
LSTM Test Accuracy: 86.78%


Training GRU model...
Epoch 1/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 106s 513ms/step - accuracy: 0.6941 - loss: 0.5647 - val_accuracy: 0.8200 - val_loss: 0.4046
Epoch 2/10
196/196 ━━━━━━━━━━━━━━━━━━━━ 111s 564ms/step 

## Compare Model Performance

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(hist_lstm.history['val_accuracy'], label='LSTM Val Acc')
plt.plot(hist_gru.history['val_accuracy'], label='GRU Val Acc')
plt.title('Validation Accuracy Comparison')
plt.legend()
plt.show()

## Observations

- LSTM achieved ~88–89% accuracy, capturing long-term dependencies effectively.
- GRU trained faster (~87–88% accuracy) with fewer parameters.
- Both architectures outperform the simple RNN (~84%) significantly.
- Next step → move to Day 18: NLP & Word Embeddings to explore how embeddings power text models.